# Naver Blog Review Preprocessing

# 목표1: 문장 분리!!

## 사용 라이브러리 설치

In [1]:
# 한글 문장 분리기 
!pip install kss # korean sentence splitter

In [2]:
# 한글 맞춤법 검사기
!pip install requests # py-hanspell 의존 라이브러리
!pip install git+https://github.com/ssut/py-hanspell.git # py-hanspell
!pip install py-hangul-checker # py-hangul-checker

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-w0ka_7a6
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-w0ka_7a6


In [3]:
# 한글 띄어쓰기 교정기 
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-9p5dv2oj
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-9p5dv2oj
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


[soynlp](https://github.com/lovit/soynlp)

In [4]:
# 고유명사 추출에 용이한 토크나이저
!pip install soynlp

## 파일 경로 설정 

In [5]:
!mkdir -p ~/jungcheck/DataPreprocessing    
!pwd    
!cd ~/jungcheck    
!mkdir -p ./data/google_review    
!mkdir -p ./data/naver_review  

/home/aiffel-dj19/jungcheck/DataPreprocessing


In [6]:
# 현재 경로 확인 
!pwd

/home/aiffel-dj19/jungcheck/DataPreprocessing


아래 tree 명령어로 상위 폴더(jungcheck)내 파일 구성을 볼 수 있습니다.    

In [7]:
# jungcheck 폴더내 경로 확인 
!tree -L 3 -N .

.
├── data
│   ├── google_review
│   │   ├── 한빛탑.csv
│   │   ├── 엑스포다리.csv
│   │   ├── 엑스포음악분수.csv
│   │   ├── 엑스포과학공원.csv
│   │   └── 세계엑스포기념품박물관.csv
│   └── naver_review
│       ├── 오월드 관광 대전.csv
│       ├── 오월드 여행 후기.csv
│       ├── 오월드 여행 대전.csv
│       ├── 오월드 관광 주말에.csv
│       ├── 오월드 여행 주말에.csv
│       ├── 뿌리공원 여행 후기.csv
│       ├── 뿌리공원 관광 후기.csv
│       ├── 뿌리공원 관광 대전.csv
│       ├── 뿌리공원 여행 대전.csv
│       ├── 뿌리공원 여행 주말에.csv
│       ├── 뿌리공원 관광 주말에.csv
│       ├── 엑스포과학공원 여행 대전.csv
│       └── 엑스포과학공원 여행 후기.csv
├── filtered
│   ├── google_review
│   │   └── google_expo_sample.csv
│   └── naver_review
├── googleReviewPreprocessing.ipynb
├── naverReviewPreprocessing.ipynb
├── naverReviewPreprocessingV0.ipynb
└── reference
    └── Comparative Study of Tokenizer Based on Learning for Sentiment Analysis.pdf

7 directories, 23 files


## Data Load 

In [8]:
import os 
import pandas as pd 
import glob

In [9]:
data_path = os.path.join(os.getcwd(), 'data/naver_review/')
data_path

'/home/aiffel-dj19/jungcheck/DataPreprocessing/data/naver_review/'

In [10]:
files = os.listdir(data_path)
files

['오월드 관광 주말에.csv',
 '오월드 관광 대전.csv',
 '엑스포과학공원 여행 대전.csv',
 '뿌리공원 여행 주말에.csv',
 '오월드 여행 주말에.csv',
 '뿌리공원 관광 주말에.csv',
 '오월드 여행 후기.csv',
 '뿌리공원 여행 후기.csv',
 '뿌리공원 관광 후기.csv',
 '뿌리공원 관광 대전.csv',
 '뿌리공원 여행 대전.csv',
 '엑스포과학공원 여행 후기.csv',
 '오월드 여행 대전.csv']

In [11]:
test_file = os.path.join(data_path, files[0])
test_file

'/home/aiffel-dj19/jungcheck/DataPreprocessing/data/naver_review/오월드 관광 주말에.csv'

In [12]:
target = '오월드'

In [13]:
target_list = []

for file_name in files:
    if target in file_name:
        target_list.append(file_name)
target_list

['오월드 관광 주말에.csv',
 '오월드 관광 대전.csv',
 '오월드 여행 주말에.csv',
 '오월드 여행 후기.csv',
 '오월드 여행 대전.csv']

## Data 확인

네이버 블로그 리뷰는 글이 길기 때문에 pandas의 option을 변경해줘야합니다. 

In [14]:
# 전체 option 확인
# pd.describe_option()

In [15]:
# get pandas option 
print(pd.get_option('display.max_rows'))
print(pd.get_option('display.max_columns'))
print(pd.get_option('display.expand_frame_repr'))

60
20
True


In [16]:
# set pandas option 
pd.set_option('display.max_rows', None) # 전체 행 보기
pd.set_option('display.max_columns', None) # 전체 열 보기 
pd.set_option('display.expand_frame_repr', False)

In [17]:
test_df = pd.read_csv(test_file, names=['date', 'description', 'text', 'name'], encoding='utf-8')  # csv 파일 읽기
len(test_df)

126

In [18]:
# test_df    # 전체 내용 보기
test_df.head() # 앞에 5행만 보기

,date,description,text,name
0,17.06.22,대전관광명소 오월드 튤립축제&amp; 사파리투어 대전관광명소 중 아이들이 가장 좋아...,\n\n대전관광명소오월드 튤립축제& 사파리투어대전관광명소 중 아이들이 가장 좋아할 ...,친절한 혁명가의 tiNY house
1,17.09.28,오월드 전국에서 오는 동물원중 하나입니다. 주말에 특히 사람이 붐비는데요 주차시설 ...,\n\n안녕하세요추석도 얼마 남지않앗네요즐거운 명절 보내시기 바랍니다이번 매물은 오...,올레공인중개사무소
2,15.10.19,"두 꼬밍이들 기어코 킥보드를 끌고 나온다규ㅡ.,ㅡ #대전동물원 #오월드 입구에보면,...","\n\n몇주전부터 가려고했으나,날씨와 시간이 협조해주지않은 관계로 미뤄진동물원 나들...",beautiful my life~♡
3,14.11.02,평소 주말은 사람들이 너무 많아 대기시간도 길고 복잡하죠ㅠ 아점을 먹고 간식거리 챙...,"\n\n비오는날 놀이공원, 동물원/ 대전 오월드\n \n \n전날 일기예보를 보니 ...",공존센터
4,17.04.14,"이번 주말에는 화사한 봄기운이 가득한 오월드 튤립축제를 방문해보시는 건, 어떨까요?...",\n\n 거리 곳곳 활짝 핀 봄꽃들을 보면 절로 미소가 지어지는 계절인데요. 이번 ...,대전시인터넷방송 공식블로그


In [19]:
origin_list = test_df.text[0]
print(len(test_df.text[0])) # 문자 개수
print(test_df.text[0]) # 내용 확인

2206


대전관광명소오월드 튤립축제& 사파리투어대전관광명소 중 아이들이 가장 좋아할 만한 장소는 아마 오월드가 아닐까 싶습니다.특히 봄에는 다양한 플라워 축제로 어른들까지 만족시켜줄 수 있는 장소였어요.4월에 있었던 오월드 튤립축제에는 주말이라 그런지 사람들이 차~암 많더라고요.꽃구경하러 겸사겸사 가긴 했었지만알록달록 놀이기구들을 보자마자 흥분하던 아이들.그러나 아이들이 뭔가를 타고 싶어해도 줄을 많이 서야 하니 아주 골치가...그래도 아침일찍 도착을 한 덕분에 서너가지 정도는 적당히 줄을 서고 아이들에게 놀이기구를 태워줄 수가 있었습니다.할머니, 할아버지가 곁에서 봐주시며 손을 흔들어 주시고계셔서 더욱 즐거웠던 아이들.놀이기구만 몇 개 태워줬는데 어느새 점심시간이 돼버렸어요.아이들 챙기랴~ 부모님 챙기랴~ㅎㅎㅎ오월드를 잘 즐기는 방법에 대한 팁이 별로 없던 상황에놀이기구 줄을 서면서 옆에 같이 줄을 서던 다른 엄마 덕에 알게 된 사실은??오월드에서는 사파리투어는 꼭 하고 가야 한다는...아~~~그렇군요!열심히 마지막으로 줄을 또 한번 서서 부모님과 아이들을 위해 사파리 투어까지 하게 되었어요.동물모양 예쁜 사파리투어 버스를 타고 부모님과 아이들을 자리에 앉히고 나니 다들 기대감 폭발~새로운 동물들을 하나하나 만날 때마다 반가운 함성이 여기저기서 터져 나옵니다.우리 아이들도 사파리투어는 처음이었지만연로하신 부모님에게는 더 신기한 체험이 되었던 시간이에요.마치 아이들을 보는 듯, 백두산 호랑이 우리 앞에서 크게 감탄을 하시기도 하고 원숭이들의 빠른 몸짓에귀엽다며 껄껄 웃으시기도 하시던 부모님.하지만 원래 목적이었던 튤립축제를 보러 가기 위해서는 고난이 아주 많았습니다.부모님을 휠체어로 모시고 플라워 랜드까지 가는 길이 대부분 언덕을 올라가듯 경사가 가파른 길밖에는 없어서 말이에요... ㅠ.ㅠ 동물원 쪽으로 휠체어를 밀고 올라갈 때에도 기나긴 비탈 길을 낑낑거리며 올라가던 내가 안쓰러웠는지 중간에 힘 좋으신 몇 분이 도와주시기도 하셨었지요. 하지만 도저히 플라

In [20]:
type(test_df.text[0])

str

## 문장 분리 

토크나이저를 활용해 'text'에 해당하는 네이버 블로그 본문에 대한 문장 분리를 진행해줍니다. 이러한 문장 분리 관련한 자연어 처리를 Sentence boundary disambiguation(SBD)라고 합니다. 해당 키워드로 검색하면 관련한 자료들이 많이 나옵니다.
* [kss(korean sentence splitter)](http://docs.likejazz.com/kss/) 활용 => 한글 SBD 해결에 유용
* [spaCy](https://www.facebook.com/groups/TensorFlowKR/permalink/1025310851143306/) => 영어 SBD 해결에 유용

In [21]:
import kss 

In [22]:
# 문장에 마침표 없는 경우 어떻게 문장 분리 수행하는지 확인
test = ['과연 어떻게 할지 궁금합니다 이거는 마침표없는 문장인데 어떻게 분리해낼까 할 수 있을까나 걱정됩니다']
test = list(map(kss.split_sentences, test)) # 문장으로 split
test

[['과연 어떻게 할지 궁금합니다', '이거는 마침표없는 문장인데 어떻게 분리해낼까 할 수 있을까나 걱정됩니다']]

In [23]:
# 문장에 마침표 없는 경우 어떻게 문장 분리 수행하는지 확인
test = ['과연 어떻게 할지 궁금하다 이거는 마침표없는 문장인데 어떻게 분리해낼까 할 수 있을까나 걱정됩니다']
test = list(map(kss.split_sentences, test)) # 문장으로 split
test

[['과연 어떻게 할지 궁금하다 이거는 마침표없는 문장인데 어떻게 분리해낼까 할 수 있을까나 걱정됩니다']]

In [24]:
test_df['text'] = test_df['text'].map(kss.split_sentences) # dataframe 문장으로 split
# test_list = list(map(kss.split_sentences, test_df['text'][0])) # list로 구성된 문장으로 split

In [25]:
test_df['text'][0]
# test_list

['대전관광명소오월드 튤립축제& 사파리투어대전관광명소 중 아이들이 가장 좋아할 만한 장소는 아마\xa0오월드가 아닐까 싶습니다.',
 '특히 봄에는 다양한\xa0플라워 축제로 어른들까지 만족시켜줄 수 있는 장소였어요.',
 '4월에 있었던 오월드 튤립축제에는 주말이라 그런지\xa0사람들이 차~암 많더라고요.',
 '꽃구경하러 겸사겸사 가긴 했었지만알록달록 놀이기구들을 보자마자 흥분하던 아이들.그러나 아이들이 뭔가를 타고 싶어해도 줄을 많이 서야 하니 아주 골치가...',
 '그래도 아침일찍 도착을 한 덕분에 서너가지 정도는\xa0적당히 줄을 서고 아이들에게 놀이기구를 태워줄 수가 있었습니다.',
 '할머니, 할아버지가 곁에서 봐주시며 손을 흔들어 주시고계셔서 더욱 즐거웠던 아이들.놀이기구만 몇 개 태워줬는데 어느새 점심시간이 돼버렸어요.',
 '아이들 챙기랴~ 부모님 챙기랴~ㅎㅎㅎ오월드를 잘 즐기는 방법에 대한 팁이 별로 없던 상황에놀이기구 줄을 서면서 옆에 같이 줄을 서던 다른 엄마 덕에 알게 된 사실은??오월드에서는 사파리투어는 꼭 하고 가야 한다는...아~~~그렇군요!',
 '열심히 마지막으로 줄을 또 한번 서서 부모님과 아이들을 위해 사파리 투어까지 하게 되었어요.',
 '동물모양 예쁜 사파리투어 버스를 타고 부모님과 아이들을\xa0자리에 앉히고 나니 다들 기대감 폭발~새로운 동물들을 하나하나 만날 때마다 반가운 함성이 여기저기서 터져 나옵니다.',
 '우리 아이들도 사파리투어는 처음이었지만연로하신 부모님에게는 더\xa0신기한 체험이 되었던 시간이에요.',
 '마치 아이들을 보는 듯, 백두산 호랑이 우리 앞에서\xa0크게 감탄을 하시기도 하고\xa0원숭이들의 빠른 몸짓에귀엽다며 껄껄 웃으시기도 하시던 부모님.하지만 원래 목적이었던 튤립축제를 보러 가기 위해서는 고난이 아주 많았습니다.',
 '부모님을 휠체어로 모시고 플라워 랜드까지 가는 길이\xa0대부분 언덕을 올라가듯 경사가 가파른 길밖에는 없어서 말이에요... ㅠ.ㅠ',
 '동물원 쪽으로\xa0휠체

In [31]:
# 문장별로 확인 
test_list = test_df['text'][0]
print(len(test_list)) # 문장 개수

for i in range(len(test_list)): # 내용 확인
    print(f'{i}번째 문장: {test_list[i]}')

29
0번째 문장: 대전관광명소오월드 튤립축제& 사파리투어대전관광명소 중 아이들이 가장 좋아할 만한 장소는 아마 오월드가 아닐까 싶습니다.
1번째 문장: 특히 봄에는 다양한 플라워 축제로 어른들까지 만족시켜줄 수 있는 장소였어요.
2번째 문장: 4월에 있었던 오월드 튤립축제에는 주말이라 그런지 사람들이 차~암 많더라고요.
3번째 문장: 꽃구경하러 겸사겸사 가긴 했었지만알록달록 놀이기구들을 보자마자 흥분하던 아이들.그러나 아이들이 뭔가를 타고 싶어해도 줄을 많이 서야 하니 아주 골치가...
4번째 문장: 그래도 아침일찍 도착을 한 덕분에 서너가지 정도는 적당히 줄을 서고 아이들에게 놀이기구를 태워줄 수가 있었습니다.
5번째 문장: 할머니, 할아버지가 곁에서 봐주시며 손을 흔들어 주시고계셔서 더욱 즐거웠던 아이들.놀이기구만 몇 개 태워줬는데 어느새 점심시간이 돼버렸어요.
6번째 문장: 아이들 챙기랴~ 부모님 챙기랴~ㅎㅎㅎ오월드를 잘 즐기는 방법에 대한 팁이 별로 없던 상황에놀이기구 줄을 서면서 옆에 같이 줄을 서던 다른 엄마 덕에 알게 된 사실은??오월드에서는 사파리투어는 꼭 하고 가야 한다는...아~~~그렇군요!
7번째 문장: 열심히 마지막으로 줄을 또 한번 서서 부모님과 아이들을 위해 사파리 투어까지 하게 되었어요.
8번째 문장: 동물모양 예쁜 사파리투어 버스를 타고 부모님과 아이들을 자리에 앉히고 나니 다들 기대감 폭발~새로운 동물들을 하나하나 만날 때마다 반가운 함성이 여기저기서 터져 나옵니다.
9번째 문장: 우리 아이들도 사파리투어는 처음이었지만연로하신 부모님에게는 더 신기한 체험이 되었던 시간이에요.
10번째 문장: 마치 아이들을 보는 듯, 백두산 호랑이 우리 앞에서 크게 감탄을 하시기도 하고 원숭이들의 빠른 몸짓에귀엽다며 껄껄 웃으시기도 하시던 부모님.하지만 원래 목적이었던 튤립축제를 보러 가기 위해서는 고난이 아주 많았습니다.
11번째 문장: 부모님을 휠체어로 모시고 플라워 랜드까지 가는 길이 대부분 언덕을 올라가듯 경사가 가파른 길밖에는 없어서 말

kss는 마침표가 없으면 한 문으로 인식하는 문제점이 있습니다. 그래서 normalize를 시도해서 한번 문장 정제를 거치고 split해봤습니다. 해도 안될시에는 [hanspell 라이브러리](https://ebbnflow.tistory.com/246)를 사용해볼 예정입니다.

## 문단 정규표현식

* [정규표현식 설명 블로그](https://blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221347960543)

In [32]:
import re

In [33]:
def senPreprocessing(sen):
    sen = re.sub(pattern='[-=+,#/\:^$@*\"※~&%ㆍ』\\‘|\(\)\[\]\<\>`\'…》]', repl=' ', string=sen)
    sen = re.sub(pattern='[\n\xa0]', repl=' ', string=sen) 
    sen = re.sub(pattern='[ㄱ-ㅎㅏ-ㅣ]+', repl=' ', string=sen) 
    sen = re.sub(pattern='[?!~.]?', repl='', string=sen) 
    sen = re.sub(pattern='[ ]', repl='', string=sen) 
#     sen = re.sub(pattern='^[?]+', repl='', string=sen) 
#     sen = re.sub(pattern='^[!]+', repl='', string=sen) 
#     sen = re.sub(pattern='^[~]+', repl='', string=sen) 
    return sen

이후에 맞춤법 교정에서 띄어쓰기를 교정할때 문장이 애매하게 띄어쓰기 되어있는 것보다, 아예 이어붙어진 경우가 교정이 더 잘 됩니다.

In [34]:
test_list = list(map(senPreprocessing, test_list))

In [35]:
test_list

['대전관광명소오월드튤립축제사파리투어대전관광명소중아이들이가장좋아할만한장소는아마오월드가아닐까싶습니다',
 '특히봄에는다양한플라워축제로어른들까지만족시켜줄수있는장소였어요',
 '4월에있었던오월드튤립축제에는주말이라그런지사람들이차암많더라고요',
 '꽃구경하러겸사겸사가긴했었지만알록달록놀이기구들을보자마자흥분하던아이들그러나아이들이뭔가를타고싶어해도줄을많이서야하니아주골치가',
 '그래도아침일찍도착을한덕분에서너가지정도는적당히줄을서고아이들에게놀이기구를태워줄수가있었습니다',
 '할머니할아버지가곁에서봐주시며손을흔들어주시고계셔서더욱즐거웠던아이들놀이기구만몇개태워줬는데어느새점심시간이돼버렸어요',
 '아이들챙기랴부모님챙기랴오월드를잘즐기는방법에대한팁이별로없던상황에놀이기구줄을서면서옆에같이줄을서던다른엄마덕에알게된사실은오월드에서는사파리투어는꼭하고가야한다는아그렇군요',
 '열심히마지막으로줄을또한번서서부모님과아이들을위해사파리투어까지하게되었어요',
 '동물모양예쁜사파리투어버스를타고부모님과아이들을자리에앉히고나니다들기대감폭발새로운동물들을하나하나만날때마다반가운함성이여기저기서터져나옵니다',
 '우리아이들도사파리투어는처음이었지만연로하신부모님에게는더신기한체험이되었던시간이에요',
 '마치아이들을보는듯백두산호랑이우리앞에서크게감탄을하시기도하고원숭이들의빠른몸짓에귀엽다며껄껄웃으시기도하시던부모님하지만원래목적이었던튤립축제를보러가기위해서는고난이아주많았습니다',
 '부모님을휠체어로모시고플라워랜드까지가는길이대부분언덕을올라가듯경사가가파른길밖에는없어서말이에요',
 '동물원쪽으로휠체어를밀고올라갈때에도기나긴비탈길을낑낑거리며올라가던내가안쓰러웠는지중간에힘좋으신몇분이도와주시기도하셨었지요',
 '하지만도저히플라워랜드까지는무리라며주변사람들이다고개를절레절레하지만원래목적이부모님꽃구경시켜드릴목적이었는데너무아쉽기만했어요등줄기에서는땀이줄줄흐르고있고자신은별로없었지만고민고민하다가결국부딪혀보자',
 '뜻이있으면길이있다플라',
 '워랜드에드디어',
 '도착했습니',
 '다',
 '예쁜꽃밭들을마주하면서아이들표정도한껏밝아지기시작하네요',
 '물론중간중간땅이고르지못해여

In [36]:
# 맺음말?에 해당하는 경우 .를 붙여줍니다.
sent ='그랬습니다' 
if sent[-1] in ['다', '요', '죠', '오', '자']:
    print(sent + '.')

그랬습니다.


In [39]:
def dotAdd(sen):
    if sen[-1] in ['다', '요', '죠', '오', '자']:
        sen = sen + '.'
    return sen

In [40]:
test_list = list(map(dotAdd, test_list))
test_list

['대전관광명소오월드튤립축제사파리투어대전관광명소중아이들이가장좋아할만한장소는아마오월드가아닐까싶습니다.',
 '특히봄에는다양한플라워축제로어른들까지만족시켜줄수있는장소였어요.',
 '4월에있었던오월드튤립축제에는주말이라그런지사람들이차암많더라고요.',
 '꽃구경하러겸사겸사가긴했었지만알록달록놀이기구들을보자마자흥분하던아이들그러나아이들이뭔가를타고싶어해도줄을많이서야하니아주골치가',
 '그래도아침일찍도착을한덕분에서너가지정도는적당히줄을서고아이들에게놀이기구를태워줄수가있었습니다.',
 '할머니할아버지가곁에서봐주시며손을흔들어주시고계셔서더욱즐거웠던아이들놀이기구만몇개태워줬는데어느새점심시간이돼버렸어요.',
 '아이들챙기랴부모님챙기랴오월드를잘즐기는방법에대한팁이별로없던상황에놀이기구줄을서면서옆에같이줄을서던다른엄마덕에알게된사실은오월드에서는사파리투어는꼭하고가야한다는아그렇군요.',
 '열심히마지막으로줄을또한번서서부모님과아이들을위해사파리투어까지하게되었어요.',
 '동물모양예쁜사파리투어버스를타고부모님과아이들을자리에앉히고나니다들기대감폭발새로운동물들을하나하나만날때마다반가운함성이여기저기서터져나옵니다.',
 '우리아이들도사파리투어는처음이었지만연로하신부모님에게는더신기한체험이되었던시간이에요.',
 '마치아이들을보는듯백두산호랑이우리앞에서크게감탄을하시기도하고원숭이들의빠른몸짓에귀엽다며껄껄웃으시기도하시던부모님하지만원래목적이었던튤립축제를보러가기위해서는고난이아주많았습니다.',
 '부모님을휠체어로모시고플라워랜드까지가는길이대부분언덕을올라가듯경사가가파른길밖에는없어서말이에요.',
 '동물원쪽으로휠체어를밀고올라갈때에도기나긴비탈길을낑낑거리며올라가던내가안쓰러웠는지중간에힘좋으신몇분이도와주시기도하셨었지요.',
 '하지만도저히플라워랜드까지는무리라며주변사람들이다고개를절레절레하지만원래목적이부모님꽃구경시켜드릴목적이었는데너무아쉽기만했어요등줄기에서는땀이줄줄흐르고있고자신은별로없었지만고민고민하다가결국부딪혀보자.',
 '뜻이있으면길이있다플라',
 '워랜드에드디어',
 '도착했습니',
 '다.',
 '예쁜꽃밭들을마주하면서아이들표정도한껏밝아지기시작하네요.',
 

In [41]:
# 문장들을 다시 한 문단으로 만들어줍니다.
test_list = ' '.join(test_list)
test_list

'대전관광명소오월드튤립축제사파리투어대전관광명소중아이들이가장좋아할만한장소는아마오월드가아닐까싶습니다. 특히봄에는다양한플라워축제로어른들까지만족시켜줄수있는장소였어요. 4월에있었던오월드튤립축제에는주말이라그런지사람들이차암많더라고요. 꽃구경하러겸사겸사가긴했었지만알록달록놀이기구들을보자마자흥분하던아이들그러나아이들이뭔가를타고싶어해도줄을많이서야하니아주골치가 그래도아침일찍도착을한덕분에서너가지정도는적당히줄을서고아이들에게놀이기구를태워줄수가있었습니다. 할머니할아버지가곁에서봐주시며손을흔들어주시고계셔서더욱즐거웠던아이들놀이기구만몇개태워줬는데어느새점심시간이돼버렸어요. 아이들챙기랴부모님챙기랴오월드를잘즐기는방법에대한팁이별로없던상황에놀이기구줄을서면서옆에같이줄을서던다른엄마덕에알게된사실은오월드에서는사파리투어는꼭하고가야한다는아그렇군요. 열심히마지막으로줄을또한번서서부모님과아이들을위해사파리투어까지하게되었어요. 동물모양예쁜사파리투어버스를타고부모님과아이들을자리에앉히고나니다들기대감폭발새로운동물들을하나하나만날때마다반가운함성이여기저기서터져나옵니다. 우리아이들도사파리투어는처음이었지만연로하신부모님에게는더신기한체험이되었던시간이에요. 마치아이들을보는듯백두산호랑이우리앞에서크게감탄을하시기도하고원숭이들의빠른몸짓에귀엽다며껄껄웃으시기도하시던부모님하지만원래목적이었던튤립축제를보러가기위해서는고난이아주많았습니다. 부모님을휠체어로모시고플라워랜드까지가는길이대부분언덕을올라가듯경사가가파른길밖에는없어서말이에요. 동물원쪽으로휠체어를밀고올라갈때에도기나긴비탈길을낑낑거리며올라가던내가안쓰러웠는지중간에힘좋으신몇분이도와주시기도하셨었지요. 하지만도저히플라워랜드까지는무리라며주변사람들이다고개를절레절레하지만원래목적이부모님꽃구경시켜드릴목적이었는데너무아쉽기만했어요등줄기에서는땀이줄줄흐르고있고자신은별로없었지만고민고민하다가결국부딪혀보자. 뜻이있으면길이있다플라 워랜드에드디어 도착했습니 다. 예쁜꽃밭들을마주하면서아이들표정도한껏밝아지기시작하네요. 물론중간중간땅이고르지못해여러번휠체어를아이들이지키고서있어야만했고빈번하게화장실을가셔야하는아버님사정때문에시간이많이지체가되기도했었지요. 하지만고

다시 문장 분리 수행

In [42]:
# test_list = list(map(kss.split_sentences, test_list)) # list로 구성된 문장으로 split
test_list = kss.split_sentences(test_list)
test_list

['대전관광명소오월드튤립축제사파리투어대전관광명소중아이들이가장좋아할만한장소는아마오월드가아닐까싶습니다.',
 '특히봄에는다양한플라워축제로어른들까지만족시켜줄수있는장소였어요.',
 '4월에있었던오월드튤립축제에는주말이라그런지사람들이차암많더라고요.',
 '꽃구경하러겸사겸사가긴했었지만알록달록놀이기구들을보자마자흥분하던아이들그러나아이들이뭔가를타고싶어해도줄을많이서야하니아주골치가 그래도아침일찍도착을한덕분에서너가지정도는적당히줄을서고아이들에게놀이기구를태워줄수가있었습니다.',
 '할머니할아버지가곁에서봐주시며손을흔들어주시고계셔서더욱즐거웠던아이들놀이기구만몇개태워줬는데어느새점심시간이돼버렸어요.',
 '아이들챙기랴부모님챙기랴오월드를잘즐기는방법에대한팁이별로없던상황에놀이기구줄을서면서옆에같이줄을서던다른엄마덕에알게된사실은오월드에서는사파리투어는꼭하고가야한다는아그렇군요.',
 '열심히마지막으로줄을또한번서서부모님과아이들을위해사파리투어까지하게되었어요.',
 '동물모양예쁜사파리투어버스를타고부모님과아이들을자리에앉히고나니다들기대감폭발새로운동물들을하나하나만날때마다반가운함성이여기저기서터져나옵니다.',
 '우리아이들도사파리투어는처음이었지만연로하신부모님에게는더신기한체험이되었던시간이에요.',
 '마치아이들을보는듯백두산호랑이우리앞에서크게감탄을하시기도하고원숭이들의빠른몸짓에귀엽다며껄껄웃으시기도하시던부모님하지만원래목적이었던튤립축제를보러가기위해서는고난이아주많았습니다.',
 '부모님을휠체어로모시고플라워랜드까지가는길이대부분언덕을올라가듯경사가가파른길밖에는없어서말이에요.',
 '동물원쪽으로휠체어를밀고올라갈때에도기나긴비탈길을낑낑거리며올라가던내가안쓰러웠는지중간에힘좋으신몇분이도와주시기도하셨었지요.',
 '하지만도저히플라워랜드까지는무리라며주변사람들이다고개를절레절레하지만원래목적이부모님꽃구경시켜드릴목적이었는데너무아쉽기만했어요등줄기에서는땀이줄줄흐르고있고자신은별로없었지만고민고민하다가결국부딪혀보자.',
 '뜻이있으면길이있다플라 워랜드에드디어 도착했습니 다.',
 '예쁜꽃밭들을마주하면서아이들표정도한껏밝아지기시작하네요.',
 '물론중간중간땅이고르지못해여러

문장분리 > 정규표현식 > 맞춤법 검사 > 다시 문단 > 문장분리를 수행했을때 이상하게 문장 중간에 짤리는 경우가 개선된 것을 확인할 수 있습니다. 마지막으로 문장 분리 후 다시 맞춤법 검사를 거치면 정제된 문장이 추출되는 것을 확인할 수 있습니다.

In [43]:
# pattern = '(http|tfp|https)://(?:[\w.]|(?:%[\da-fA-F\{2}+' # html 링크 
# ^[가-힣a-zA-Z0-9]+$ # 영문, 한글, 숫자만 가능 

## 맞춤법 검사 후 문장 분리
* [py-hanspell](https://github.com/ssut/py-hanspell): 네이버 맞춤법 검사기를 기반으로 합니다, pip 설치시 error가 있어서 git clone을 활용해야합니다. 
* [부산대 맞춤법 검사기 using selenium](https://hong-yp-ml-records.tistory.com/99): 부산대 맞춤법 검사기는 python 2.x 버전에 해당해서, 해당 블로그에서 Selenium을 활용해 맞춤법 검사를 진행한다. 해당 블로그 실험에서는 py-hanspell의 성능이 나았으므로 맞춤법 검사는 py-hanspell로만 진행하겠습니다. 
* [py-hangul-checker](https://pypi.org/project/py-hangul-checker/)

### py-hangul-check를 활용한 맞춤법 검사

In [44]:
from hangul_checker import KoreanSpellChecker

In [45]:
ksc = KoreanSpellChecker()
# test_df['text'] = test_df['text'].map(ksc.check_spelling)

In [46]:
# 특수 기호 제거하지 않고 맞춤법 검사 수행 
# test_list = test_df['text'][1]
print(len(test_list)) # 문장 개수

for i in range(len(test_list)): # 내용 확인
    print(f'맞춤법 교정전 {i}번째 문장: {test_list[i]}')
    test_list[i] = ksc.check_spelling(test_list[i])
    print(f'맞춤법 교정후 {i}번째 문장: {test_list[i]}')
    print('-'*40)

23
맞춤법 교정전 0번째 문장: 대전관광명소오월드튤립축제사파리투어대전관광명소중아이들이가장좋아할만한장소는아마오월드가아닐까싶습니다.
맞춤법 교정후 0번째 문장: 대전관광명소 오월드 튤립축제 사파리 투어 대전관광명소 중 아이들이 가장 좋아할 만한 장소는 아마 오월드가 아닐까 싶습니다.
----------------------------------------
맞춤법 교정전 1번째 문장: 특히봄에는다양한플라워축제로어른들까지만족시켜줄수있는장소였어요.
맞춤법 교정후 1번째 문장: 특히 봄에는 다양한 플라워 축제로 어른들까지 만족시켜 줄 수 있는 장소였어요.
----------------------------------------
맞춤법 교정전 2번째 문장: 4월에있었던오월드튤립축제에는주말이라그런지사람들이차암많더라고요.
맞춤법 교정후 2번째 문장: 4월에 있었던 오월드 튤립축제에는 주말이라 그런지 사람들이 차암 많더라고요.
----------------------------------------
맞춤법 교정전 3번째 문장: 꽃구경하러겸사겸사가긴했었지만알록달록놀이기구들을보자마자흥분하던아이들그러나아이들이뭔가를타고싶어해도줄을많이서야하니아주골치가 그래도아침일찍도착을한덕분에서너가지정도는적당히줄을서고아이들에게놀이기구를태워줄수가있었습니다.
맞춤법 교정후 3번째 문장: 꽃구경하러 겸사겸사 가긴 했었지만 알록달록 놀이 기고들을 보자마자 흥분하던 아이들 그러나 아이들이 뭔가를 타고 싶어 해도 줄을 많이 서야 하니 아주 골치가 그래도 아침 일찍 도착을 한 덕분에 서너 가지 정도는 적당히 줄을 서고 아이들에게 놀이기구를 태워줄 수가 있었습니다.
----------------------------------------
맞춤법 교정전 4번째 문장: 할머니할아버지가곁에서봐주시며손을흔들어주시고계셔서더욱즐거웠던아이들놀이기구만몇개태워줬는데어느새점심시간이돼버렸어요.
맞춤법 교정후 4번째 문장: 할머니 할아버지가 곁에서 봐주시며 손을 흔들어주시고 계셔서 더욱 즐거웠던 아이들 놀이기구만 몇 개 태워줬는데 어느

### hanspell을 활용한 맞춤법 검사

In [47]:
from tqdm import tqdm
from hanspell import spell_checker

In [55]:
len(test_list)

23

In [62]:
result = spell_checker.check(pd.Series(test_list[0])) 
result.checked

'대전관광명소 오월드 튤립축제 사파리 투어 대전관광명소 중 아이들이 가장 좋아할 만한 장소는 아마 오월드가 아닐까 싶습니다.'

In [66]:
original_list = test_list

for i in tqdm(range(len(test_list))): 
    try: 
        result = spell_checker.check(test_list[i]) 
        test_list[i] = result.checked 
    except: 
        print('Error')
        pass 

100%|██████████| 23/23 [00:02<00:00,  9.10it/s]


In [64]:
test_list

['대전관광명소 오월드 튤립축제 사파리 투어 대전관광명소 중 아이들이 가장 좋아할 만한 장소는 아마 오월드가 아닐까 싶습니다.',
 '특히 봄에는 다양한 플라워 축제로 어른들까지 만족시켜 줄 수 있는 장소였어요.',
 '4월에 있었던 오월드 튤립축제에는 주말이라 그런지 사람들이 차암 많더라고요.',
 '꽃구경하러 겸사겸사 가긴 했었지만 알록달록 놀이 기고들을 보자마자 흥분하던 아이들 그러나 아이들이 뭔가를 타고 싶어 해도 줄을 많이 서야 하니 아주 골치가 그래도 아침 일찍 도착을 한 덕분에 서너 가지 정도는 적당히 줄을 서고 아이들에게 놀이기구를 태워줄 수가 있었습니다.',
 '할머니 할아버지가 곁에서 봐주시며 손을 흔들어주시고 계셔서 더욱 즐거웠던 아이들 놀이기구만 몇 개 태워줬는데 어느새 점심시간이 돼버렸어요.',
 '아이들 챙기랴 부모님 챙기랴 오월드를 잘 즐기는 방법에 대한 팁이 별로 없던 상황에 놀이기구 줄을 서면서 옆에 같이 줄을 서던 다른 엄마 덕에 알게 된 사실은 오월드에서는 사파리 투어는 꼭 하고 가야 한다는 아 그렇군요.',
 '열심히 마지막으로 줄을 또 한 번 서서 부모님과 아이들을 위해 사파리 투어까지 하게 되었어요.',
 '동물 모양 예쁜 사파리 투어버스를 타고 부모님과 아이들을 자리에 앉히고 나니다들 기대감 폭발 새로운 동물들을 하나하나 만날 때마다 반가운 함성이 여기저기서 터져 나옵니다.',
 '우리 아이들도 사파리 투어는 처음이었지만 연로하신 부모님에게는 더 신기한 체험이 되었던 시간이에요.',
 '마치 아이들을 보는 듯 백두산 호랑이 우리 앞에서 크게 감탄을 하시기도 하고 원숭이들의 빠른 몸짓에 귀엽다며 껄껄 웃으시기도 하시던 부모님 하지만 원래 목적이었던 튤립축제를 보러 가기 위해서는 고난이 아주 많았습니다.',
 '부모님을 휠체어로 모시고 플라워랜드까지 가는 길이 대부분 언덕을 올라가듯 경사가 가파른 길밖에는 없어서 말이에요.',
 '동물원 쪽으로 휠체어를 밀고 올라갈 때에도 기나긴 비탈길을 낑낑거리며 올라가던 내가 안쓰러웠는

In [67]:
for i in tqdm(range(len(test_list))): 
    if original_list[i] != test_list[i]:
        print(test_list[i])
        print(test_list[i])

100%|██████████| 23/23 [00:00<00:00, 189899.59it/s]


맞춤법 교정전과 교정후가 모두 일치하는 것으로 볼 때, hanspell과 py-korean-checker와 동일한 결과를 수행하는 것을 알 수 있습니다. 

하지만, 이라던게 몇몇 문장에서 띄어쓰기 다시 필요하다. 

### pykospacing 띄어쓰기 교정

* [띄어쓰기 교정 관련 블로그](https://jeongukjae.github.io/posts/korean-spacing-model/)
* [kospacing](https://github.com/haven-jeon/PyKoSpacing)
* [soyspacing](https://taeguu.tistory.com/55)

In [68]:
from pykospacing import Spacing # 띄어쓰기 교정 라이브러리 

In [69]:
spacing = Spacing(rules=['하지만']) # rule을 추가해줄 수 있다. 

In [71]:
for i in tqdm(range(len(test_list))): 
    try: 
        test_list[i] = spacing(test_list[i])  
    except: 
        print('Error')
        pass 

100%|██████████| 23/23 [00:00<00:00, 28.19it/s]


In [72]:
origin_list # raw data 

'\n\n대전관광명소오월드 튤립축제& 사파리투어대전관광명소 중 아이들이 가장 좋아할 만한 장소는 아마\xa0오월드가 아닐까 싶습니다.특히 봄에는 다양한\xa0플라워 축제로 어른들까지 만족시켜줄 수 있는 장소였어요.4월에 있었던 오월드 튤립축제에는 주말이라 그런지\xa0사람들이 차~암 많더라고요.꽃구경하러 겸사겸사 가긴 했었지만알록달록 놀이기구들을 보자마자 흥분하던 아이들.그러나 아이들이 뭔가를 타고 싶어해도 줄을 많이 서야 하니 아주 골치가...그래도 아침일찍 도착을 한 덕분에 서너가지 정도는\xa0적당히 줄을 서고 아이들에게 놀이기구를 태워줄 수가 있었습니다.할머니, 할아버지가 곁에서 봐주시며 손을 흔들어 주시고계셔서 더욱 즐거웠던 아이들.놀이기구만 몇 개 태워줬는데 어느새 점심시간이 돼버렸어요.아이들 챙기랴~ 부모님 챙기랴~ㅎㅎㅎ오월드를 잘 즐기는 방법에 대한 팁이 별로 없던 상황에놀이기구 줄을 서면서 옆에 같이 줄을 서던 다른 엄마 덕에 알게 된 사실은??오월드에서는 사파리투어는 꼭 하고 가야 한다는...아~~~그렇군요!열심히 마지막으로 줄을 또 한번 서서 부모님과 아이들을 위해 사파리 투어까지 하게 되었어요.동물모양 예쁜 사파리투어 버스를 타고 부모님과 아이들을\xa0자리에 앉히고 나니 다들 기대감 폭발~새로운 동물들을 하나하나 만날 때마다 반가운 함성이 여기저기서 터져 나옵니다.우리 아이들도 사파리투어는 처음이었지만연로하신 부모님에게는 더\xa0신기한 체험이 되었던 시간이에요.마치 아이들을 보는 듯, 백두산 호랑이 우리 앞에서\xa0크게 감탄을 하시기도 하고\xa0원숭이들의 빠른 몸짓에귀엽다며 껄껄 웃으시기도 하시던 부모님.하지만 원래 목적이었던 튤립축제를 보러 가기 위해서는 고난이 아주 많았습니다.부모님을 휠체어로 모시고 플라워 랜드까지 가는 길이\xa0대부분 언덕을 올라가듯 경사가 가파른 길밖에는 없어서 말이에요... ㅠ.ㅠ\xa0동물원 쪽으로\xa0휠체어를 밀고\xa0올라갈 때에도 기나긴 비탈 길을 낑낑거리며 올라가던 내가 안쓰러웠는지\xa0중간

In [73]:
filtered_list = ' '.join(test_list) # cleansing data
filtered_list

'대전 관광명소 오월드 튤립축제 사파리 투어 대전 관광명소 중 아이들이 가장 좋아할 만한 장소는 아마 오월드가 아닐까 싶습니다. 특히 봄에는 다양한 플라워 축제로 어른들까지 만족시켜 줄 수 있는 장소였어요. 4월에 있었던 오월드 튤립축제에는 주말이라 그런지 사람들이 차암 많더라고요. 꽃구경하러 겸사 겸사 가 긴 했었지만 알록달록 놀이 기고들을 보자마자 흥분하던 아이들 그러나 아이들이 뭔가를 타고 싶어 해도 줄을 많이 서야 하니 아주 골치가 그래도 아침 일찍 도착을 한 덕분에 서너 가지 정도는 적당히 줄을 서 고 아이들에게 놀이기구를 태워 줄 수가 있었습니다. 할머니 할아버지가 곁에서 봐주시며 손을 흔들어주시고 계셔서 더욱 즐거웠던 아이들 놀이기구만 몇 개 태워줬는데 어느새 점심시간이 돼버렸어요. 아이들 챙기랴 부모님 챙기랴 오월드를 잘 즐기는 방법에 대한 팁이 별로 없던 상황에 놀이기구 줄을 서면서 옆에 같이 줄을 서던 다른 엄마 덕에 알게 된 사실은 오월드에서는 사파리 투어는 꼭 하고 가야 한다는 아 그렇군요. 열심히 마지막으로 줄을 또 한 번 서서 부모님과 아이들을 위해 사파리 투어까지 하게 되었어요. 동물 모양 예쁜 사파리 투어버스를 타고 부모님과 아이들을 자리에 앉히고 나니 다들 기대감 폭발 새로운 동물들을 하나 하나 만날 때마다 반가운 함성이 여기저기서 터져 나옵니다. 우리 아이들도 사파리 투어는 처음이었지만 연로하신 부모님에게는 더 신기한 체험이 되었던 시간이에요. 마치 아이들을 보는 듯 백두산 호랑이 우리 앞에서 크게 감탄을 하시기도 하고 원숭이들의 빠른 몸짓에 귀엽다며 껄껄 웃으시기도 하시던 부모님 하지만 원래 목적이었던 튤립축제를 보러 가 기 위해서는 고난이 아주 많았습니다. 부모님을 휠체어로 모시고 플라워랜드까지 가는 길이 대부분 언덕을 올라가듯 경사가 가파른 길밖에는 없어서 말이에요. 동물원 쪽으로 휠체어를 밀고 올라갈 때에도 기나 긴 비탈길을 낑낑거리며 올라가던 내가 안쓰러웠는지 중간에 힘 좋으신 몇 분이 도와 주시기도 하셨었지요. 하지만

### soyspacing 띄어쓰기 교정

# 목표2: 동반자 유형 추출!! 

In [74]:
companions = {
    '가족친지': ['아기', '가족', '친척', '사촌', '부모님', '시부모', '장인어른', '아버님', '어머님', '조카', '이모', '고모', '삼촌', '할아버지', '할머니', '친형', '친언니', '친누나', '친동생', '친오빠', '엄마', '아빠', '딸', '아들', '자식'],
    '친구연인': ['동기', '후배', '선배', '남자친구', '여자친구', '자기', '과동기', '찐친', '절친', '친구', '언니', '동생', '형', '오빠'],
    '직장': ['직장', '회사', '주임님', '회사동료', '사장님', '직장동료', '직장동기', '회사동기', '회사', '직장선배', '직장후배', '선임', '후임', '차장', '부장', '부장님', '차장님', '대리님'],
    '학교': ['학과', '학교', '현장견학', '현장체험', '소풍', '수학여행']
}

In [75]:
companions.keys()

dict_keys(['가족친지', '친구연인', '직장', '학교'])

## 단어 빈도수 기반해서 동반자 유형 추출

In [76]:
companion_list = []

for key, value in companions.items():
#     print(value)
    for v in value: 
        if v in filtered_list:
            print(v)
            companion_list.append(key)

print(companion_list)

부모님
아버님
할아버지
할머니
엄마
딸
['가족친지', '가족친지', '가족친지', '가족친지', '가족친지', '가족친지']


여러 종류가 언급될 수 있으므로 가장 많이 언급된 것을 해당 동반자 유형으로 취급한다.

In [77]:
from collections import Counter

freq = Counter(companion_list)
print(freq)

Counter({'가족친지': 6})


In [78]:
companion_type = max(freq.keys(), key=(lambda k: freq[k]))
companion_type

'가족친지'

## 조사 붙은 경우 고려한 동반자 유형 추출

In [79]:
# 띄어쓰기 기반으로 문단을 한 단어씩 구분
filtered_words = filtered_list.split(' ')

In [80]:
josa = ['랑', '와', '이와', '하고', '과 같이', '와 같이', '을', '를', '은', '는', '이', '가', '와 함께', '과 함께']

In [81]:
# 추가 아이디어: 갔다, 모시고, 같이 왔다, 등 여행을 간 것에 대한 동사도 같이 등장해야함

In [82]:
'와' in ['엄마와', '아빠와']

False

In [83]:
josa_list = []

for j in josa:
    for word in filtered_words:
        if j in word and word.endswith(j) and len(word) > len(j):
#             print(word)
            josa_list.append(word)
print(josa_list)

['전통정원이랑', '도와', '감상하고', '기고들을', '줄을', '도착을', '줄을', '손을', '줄을', '줄을', '줄을', '아이들을', '아이들을', '동물들을', '아이들을', '감탄을', '부모님을', '언덕을', '비탈길을', '꽃밭들을', '화장실을', '화원들을', '꽃들을', '사진을', '억을', '춤을', '확인을', '사실을', '뭔가를', '놀이기구를', '오월드를', '투어버스를', '튤립축제를', '휠체어를', '고개를', '휠체어를', '에너지를', '원피스를', '소리를', '사이를', '휠체어를', '사실은', '자신은', '수많은', '약간은', '넓은', '많은', '이야기만은', '장소는', '봄에는', '있는', '튤립축제에는', '정도는', '즐기는', '오월드에서는', '투어는', '한다는', '투어는', '부모님에게는', '보는', '위해서는', '가는', '길밖에는', '플라워랜드까지는', '줄기에서는', '하는', '온다는', '있는', '있는', '떨어지는', '추는', '분수에서는', '통과하는', '있는', '있는', '내에서는', '서비스는', '가능하다는', '있는', '하는', '유모차는', '휠체어는', '해당되는', '아이들이', '사람들이', '놀이', '아이들이', '많이', '점심시간이', '팁이', '같이', '함성이', '체험이', '호랑이', '고난이', '길이', '분이', '사람들이', '목적이', '땀이', '뜻이', '길이', '땅이', '아이들이', '시간이', '많이', '낙이', '말이', '딸아이', '꽃들이', '튤립들이', '땡땡이', '없이', '정원들뿐만이', '분수들이', '같이', '물이', '곳이', '없이', '한국이', '마음이', '오월드가', '골치가', '수가', '할아버지가', '경사가', '내가', '고민하다가', '지체가', '수가', '관리가', '서비스가', '찾아가', '휠체어가', '보수가', '배려가', '수요가', '대가', '

In [84]:
for key, value in companions.items():
#     print(value)
    for v in value: 
        if v in josa_list:
            print(v)
            companion_list.append(key)

print(companion_list)

['가족친지', '가족친지', '가족친지', '가족친지', '가족친지', '가족친지']


In [85]:
from collections import Counter

freq = Counter(companion_list)
print(freq)

Counter({'가족친지': 6})


In [86]:
companion_type = max(freq.keys(), key=(lambda k: freq[k]))
companion_type

'가족친지'

해당 동반자 유형 단어가 언급된 문장을 뽑아봐야할듯    
mecab => 갔다, 모시고 갔다 동사원형이 있는지 확인 

# 목표3: 빈도수 높은 단어 추출!!

관광지에 해당하는 단어들은 고유명사이기 때문에 해당하는 단어들을 추출하기 위해선 konlpy보다 sonlpy가 성능이 더 좋다고 알려져있습니다. konlpy는 띄어쓰기에 강한 nlp라이브러리입니다. 
* [참고 블로그](https://git.ajou.ac.kr/open-source-2018-spring/python_Korean_NLP/-/tree/master)

In [87]:
from soynlp.noun import LRNounExtractor
from soynlp.noun import LRNounExtractor_v2
from soynlp.noun import NewsNounExtractor


def get_noun_sorted_by_frequency(noun_extractor):
    """
    Description
    이미 학습되어 있는 noun_extractor의 noun을 
    frequency 내림차순으로 정렬하여 리턴함.
    """
    return_lst = list()
    nouns = noun_extractor.extract()
    for (each_noun, each_noun_info) in nouns.items():
        each_noun_dict = {
            "noun": each_noun,
            "frequency": each_noun_info.frequency, 
            "score": each_noun_info.score
        }
        return_lst.append(each_noun_dict)
    return_lst = sorted(return_lst, key=lambda x: x['frequency'], reverse=True)
    return return_lst

In [88]:
noun_extractor_dict = {
    "LRNounExtractor": LRNounExtractor(verbose=False), 
    "LRNounExtractor_v2": LRNounExtractor_v2(verbose=False), 
    "NewsNounExtractor": NewsNounExtractor(verbose=False)
}

/home/aiffel-dj19/anaconda3/envs/aiffel/lib/python3.7/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment


In [89]:
filtered_sens = kss.split_sentences(filtered_list)
print(len(filtered_sens))

24


In [90]:
for noun_ext_name, noun_ext in noun_extractor_dict.items():
    # train
    noun_ext.train(filtered_sens)
    print(f"== extractor: {noun_ext_name}")
    # noun을 빈도 내림차순으로 정렬하여 리턴
    noun_lst = get_noun_sorted_by_frequency(noun_ext)
    for noun in noun_lst[:20]:
        print(noun)
    print("--" * 50)

== extractor: LRNounExtractor
{'noun': '아이들', 'frequency': 12, 'score': 0.752168625}
{'noun': '휠체어', 'frequency': 8, 'score': 0.6755458}
{'noun': '부모님', 'frequency': 7, 'score': 0.87611575}
{'noun': '오월드', 'frequency': 7, 'score': 0.9988863333333334}
{'noun': '튤립축제', 'frequency': 4, 'score': 0.9996625}
----------------------------------------------------------------------------------------------------
== extractor: LRNounExtractor_v2
{'noun': '부모님', 'frequency': 8, 'score': 1.0}
{'noun': '오월드', 'frequency': 8, 'score': 1.0}
{'noun': '휠체어', 'frequency': 7, 'score': 1.0}
{'noun': '줄', 'frequency': 7, 'score': 0.7777777777777778}
{'noun': '아이들', 'frequency': 6, 'score': 1.0}
{'noun': '대', 'frequency': 6, 'score': 0.7142857142857143}
{'noun': '수', 'frequency': 6, 'score': 1.0}
{'noun': '튤립축제', 'frequency': 5, 'score': 1.0}
{'noun': '투어', 'frequency': 4, 'score': 0.8}
{'noun': '꽃', 'frequency': 4, 'score': 1.0}
{'noun': '플라워랜드', 'frequency': 3, 'score': 1.0}
{'noun': '목적', 'frequency': 3, '

In [91]:
for noun_dict in noun_lst:
    target_freq = 0
    if target in noun_dict['noun']:
        target_freq = noun_dict.get('frequency')
        continue
    print(noun_dict.get('noun'))
print('-'*20)
print(f'검색 관광지명인 {target}가 {target_freq}번 언급됐습니다.')

아이들
부모님
휠체어
튤립축제
놀이기구
목적
플라워랜드
꽃들
--------------------
검색 관광지명인 오월드가 8번 언급됐습니다.


단순히 단어만 보면, 방문 이유나 목적을 알기 어려움. 그래서 문장을 가져오는게 좀더 나을거 같음..? 
[관광지 설명](https://www.daejeon.go.kr/tou/sitemap.do)을 보고서